In [1]:
import warnings
import itertools
import pandas
import math
import sys
import os
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
# Add detector summary file to check scores

detector_summary_file = "../results/TSA_MA/TSA_MA_standard_scores.csv"

In [3]:
#Reading summary file
summaryDataFrame = pandas.read_csv(detector_summary_file,index_col="File")
summaryDataFrame = summaryDataFrame.sort_values(by=['Score'],ascending=False)
print(summaryDataFrame.loc[:, 'Score'])

File
realAdExchange/exchange-3_cpm_results.csv                    0.862099
realAWSCloudwatch/ec2_cpu_utilization_ac20cd.csv             0.860672
realAWSCloudwatch/ec2_network_in_257a54.csv                  0.851668
realTraffic/speed_6005.csv                                   0.842622
artificialWithAnomaly/art_daily_flatmiddle.csv               0.838069
realAWSCloudwatch/ec2_cpu_utilization_825cc2.csv             0.635319
artificialWithAnomaly/art_daily_jumpsup.csv                  0.532056
realKnownCause/cpu_utilization_asg_misconfiguration.csv      0.432805
artificialNoAnomaly/art_flatline.csv                         0.000000
realAWSCloudwatch/ec2_cpu_utilization_c6585a.csv            -0.110000
realTraffic/speed_t4013.csv                                 -0.126586
realTraffic/occupancy_t4013.csv                             -0.137389
realAWSCloudwatch/elb_request_count_8c0756.csv              -0.138449
realAWSCloudwatch/rds_cpu_utilization_e47b3b.csv            -0.138449
realAWSCloudwat

In [4]:
# select detector summary file row
detector_summary_row = "realAdExchange/exchange-3_cpm_results.csv"

# Add detector result file 
result_file = "../results/TSA_MA/realAdExchange/TSA_MA_exchange-3_cpm_results.csv"

In [5]:
if summaryDataFrame["Threshold"][detector_summary_row].size == 1:
    threshold = summaryDataFrame["Threshold"][detector_summary_row]
    TP = summaryDataFrame["TP"][detector_summary_row]
    TN = summaryDataFrame["TN"][detector_summary_row]
    FP = summaryDataFrame["FP"][detector_summary_row]
    FN = summaryDataFrame["FN"][detector_summary_row]
    total_Count = summaryDataFrame["Total_Count"][detector_summary_row]
else:
    threshold = summaryDataFrame["Threshold"][detector_summary_row][0]
    TP = summaryDataFrame["TP"][detector_summary_row][0]
    TN = summaryDataFrame["TN"][detector_summary_row][0]
    FP = summaryDataFrame["FP"][detector_summary_row][0]
    FN = summaryDataFrame["FN"][detector_summary_row][0]
    total_Count = summaryDataFrame["Total_Count"][detector_summary_row][0]

In [6]:
#Plot Result
#Reading results file 
dataframe = pandas.read_csv(result_file)
x = np.array(dataframe[list(dataframe)[0]])
value = np.array(dataframe[list(dataframe)[1]])
anomaly_score = np.array(dataframe[list(dataframe)[2]])
anomaly_label = np.array(dataframe[list(dataframe)[3]])

# reward_low_FP_score = np.array(dataframe[list(dataframe)[4]])
# reward_low_FN_score = np.array(dataframe[list(dataframe)[5]])
# standard_score = np.array(dataframe[list(dataframe)[6]])

reward_low_FP_score = np.array(dataframe[list(dataframe)[4]])
reward_low_FN_score = np.array(dataframe[list(dataframe)[4]])
standard_score = np.array(dataframe[list(dataframe)[4]])


# Plot values, anomaly score and label scaled to values
value_max = np.max(value)
trace_value = {"x": x,
         "y": value,
         "mode": 'lines',
         "name": 'Value'}

trace_anomaly_score = {"x": x,
              "y": anomaly_score*value_max,
              "mode": 'lines',
              "name": 'Anomaly score'}

trace_anomaly_label = {"x": x,
              "y": anomaly_label*value_max,
              "mode": 'lines',
              "name": 'Anomaly window'}
trace_threshold = {"x": x,
              "y": np.ones(len(x))*threshold*value_max,
              "mode": 'lines',
              "name": 'Anomaly threshold'}

traces = [trace_value,trace_anomaly_score,trace_threshold,trace_anomaly_label]
layout = dict(title = "Anomaly score : "+result_file,
              xaxis = dict(title = 'X'),
              yaxis = dict(title = 'Value')
             )
fig = dict(data=traces, layout=layout)
iplot(fig)

#plot, anomalys score, label, and result from benchmark
trace_anomaly_score = {"x": x,
              "y": anomaly_score,
              "mode": 'lines',
              "name": 'Anomaly score'}

trace_anomaly_label = {"x": x,
              "y": anomaly_label,
              "mode": 'lines',
              "name": 'Anomaly window'}

trace_threshold = {"x": x,
              "y": np.ones(len(x))*threshold,
              "mode": 'lines',
              "name": 'Anomaly threshold'}

trace_reward_low_FP_score = {"x": x,
              "y": reward_low_FP_score,
              "mode": 'lines',
              "name": 'reward_low_FP_score'}

trace_reward_low_FN_score = {"x": x,
              "y": reward_low_FN_score,
              "mode": 'lines',
              "name": 'reward_low_FN_score'}

trace_standard_score = {"x": x,
              "y": standard_score,
              "mode": 'lines',
              "name": 'standard_score'}

traces = [trace_anomaly_score,trace_threshold,trace_anomaly_label,trace_reward_low_FP_score,trace_reward_low_FN_score,trace_standard_score]
layout = dict(title = "Anomaly score : "+result_file,
              xaxis = dict(title = 'X'),
              yaxis = dict(title = 'Value')
             )
fig = dict(data=traces, layout=layout)
iplot(fig)

In [9]:
print("For result file : " + result_file)
print("Score for Rewarding low Fals Positives : "+str(np.sum(reward_low_FP_score)))
print("Score for Rewarding low Fals Negatves : "+str(np.sum(reward_low_FN_score)))
print("Standard score : "+str(np.sum(standard_score)))

For result file : ../results/TSA_MA/realAdExchange/TSA_MA_exchange-3_cpm_results.csv
Score for Rewarding low Fals Positives : 0.8620985873520001
Score for Rewarding low Fals Negatves : 0.8620985873520001
Standard score : 0.8620985873520001


In [11]:
print("For result file : " + result_file)
print("True Positive (Detected anomalies) : " + str(TP))
print("True Negative (Detected non anomalies) : " + str(TN))
print("False Positive (False alarms) : " + str(FP))
print("False Negative (Anomaly not detected) : " + str(FN))
print("Total data points : " + str(total_Count))

For result file : ../results/TSA_MA/realAdExchange/TSA_MA_exchange-3_cpm_results.csv
True Positive (Detected anomalies) : 2
True Negative (Detected non anomalies) : 1155
False Positive (False alarms) : 0
False Negative (Anomaly not detected) : 151
Total data points : 1538
